![](./sbu_logo.png)

# Image Proccessing Related Project
- **This Project is done to show some of the applications of the Linear Algebra in the future lesson**
### Written and Developed By: Mohammad Hossein Basouli, SID: 401222020

## Imports

In [2]:
import cv2
import numpy as np
from numpy import sin, cos, radians

## Image Manipulation Class Implementation

In [5]:
class ImageManip:
    
    @staticmethod
    def resize_img(img: np.ndarray, x_scale, y_scale):
        h, w, channels = img.shape # getting dimensions of the image
        new_height, new_width = int(h * y_scale), int(w * x_scale) # calculating the output dimensions
        y_scale_inv, x_scale_inv = h / new_height, w / new_width # calculating the inverse scale
        new_img = np.zeros((new_height, new_width, channels), dtype = np.uint8) # creating a new matrix correspoding to the new dimensions
        
        for i in range(new_height):
            for j in range(new_width):
                org_heigth = int(y_scale_inv * i) # original height of i
                org_width = int(x_scale_inv * j) # original width of j
                new_img[i, j] = img[org_heigth, org_width] # reflexing an invertval of pixels to the new image
        return new_img
        

    @staticmethod
    def rotate_img(img: np.ndarray, theta):
        theta = radians(theta) # converting the angle to radian
        h, w, channels = img.shape # getting dimensions of the image
        center_i, center_j = h // 2, w // 2 # calculating the center of the image
        new_img = np.zeros((h, w, channels), dtype = np.uint8) # creating a new matrix correspoding to scales
        # Definning Rotation Matrix in 2-D
        rotation_matrix = np.array([[cos(theta), -sin(theta)], 
                                    [sin(theta), cos(theta)]])
        for i in range(h):
            for j in range(w):
                y, x = i - center_i, j - center_j # calculating the corresponding vector from center
                new_j, new_i = np.matmul(rotation_matrix, [x, y]) # final vector calculated after rotation
                new_j, new_i = int(new_j + center_j), int(new_i + center_i)
                if 0 <= new_j < w - 1 and 0 <= new_i < h - 1: # checking if the final pixel doesn't exceed the screen
                    new_img[i, j] = img[new_i, new_j] # reflexing each pixel of the old img
        return new_img
        

    @staticmethod
    def blur_img(img: np.ndarray, method: int):
       h, w, channels = img.shape # shape of the image
       new_img = np.zeros((h, w, channels), dtype = np.uint8) # creating a new matrix corresponding to scales
        
       for i in range(0, h, 10):
           for j in range(0, w, 10):
               mat = img[i : i + 10, j : j + 10] # a 10 by 10 part of the image to get convolved
               
               if method == 1:
                   # Replacing the output 
                   new_img[i : i + 10, j : j + 10] = ImageManip.convolve_gaussian_kernel_1(mat) 
               elif method == 2:
                   # Replacing the output 
                   new_img[i : i + 10, j : j + 10] = ImageManip.convolve_gaussian_kernel_2(mat)
       return new_img
        
            
    @staticmethod
    def convolve_gaussian_kernel_1(mat: np.ndarray): # First Method to Sharpen an Image
        h, w, channels = mat.shape # getting dimensions of the image
        new_mat = np.zeros((h, w, channels), dtype = np.uint8) # creating a new matrix correspoding to scales
        
        center_i, center_j = h // 2, w // 2
        
        B_channel = mat[::, ::, 0] # all Blue Channels
        G_channel = mat[::, ::, 1] # all Green Channels
        R_channel = mat[::, ::, 2] # all Red Channels
        
        B_average = np.sum(B_channel, axis= None) // (h * w) # average of B-channels
        G_average = np.sum(G_channel, axis= None) // (h * w) # average of G-channels
        R_average = np.sum(R_channel, axis= None) // (h * w) # average of R-channels
        
        for i in range(h):
            for j in range(w):
                d = int((((i - center_i)/center_i) ** 2 + ((j - center_j)/center_j) ** 2) ** 0.5) # a ratio to get subtracted from average
                new_mat[i, j] = np.array([B_average - d * B_average, G_average - d * G_average, R_average - d * R_average]) # average - ratio * average
        return new_mat
        

    @staticmethod
    def convolve_gaussian_kernel_2(mat: np.ndarray): # Second Method to Sharpen an Image
        h, w, channels = mat.shape # getting dimensions of the image
        new_mat = np.zeros((h, w, channels), dtype = np.uint8) # creating a new matrix correspoding to scales
        
        center_i, center_j = h // 2, w // 2
        
        B_channel = mat[::, ::, 0] # all Blue Channels
        G_channel = mat[::, ::, 1] # all Green Channels
        R_channel = mat[::, ::, 2] # all Red Channels
        
        B_average = np.sum(B_channel, axis= None) // (h * w) # average of B-channels
        G_average = np.sum(G_channel, axis= None) // (h * w) # average of G-channels
        R_average = np.sum(R_channel, axis= None) // (h * w) # average of R-channels
        
        for i in range(h):
            for j in range(w):
                new_mat[i, j] = np.array([B_average, G_average, R_average]) # replacing each image by the average color channel of it's region
        return new_mat
        

    @staticmethod
    def sharpen_img(img: np.ndarray):
       h, w, channels = img.shape # shape of the image
       new_img = np.zeros((h - 2, w - 2, channels), dtype = np.uint8) # creating a new matrix corresponding to scales
        
       for i in range(1, h - 1):
           for j in range(1, w - 1): 
               mat = img[i - 1 : i + 2, j - 1 : j + 2] # part of image to get convolved
               new_img[i - 1, j - 1] = ImageManip.convolve_sharpen_matrix(mat) # placing in the output img
       return new_img
        

    @staticmethod
    def convolve_sharpen_matrix(mat: np.ndarray):
        # Specific Kernel to Sharpen the Image
        sharpen_kernel = np.array([[0, -1, 0], 
                                   [-1, 5, -1], 
                                   [0, -1, 0]])
        
        accumulator_b = 0 # acumulator for blue channel
        accumulator_g = 0 # acumulator for green channel
        accumulator_r = 0 # acumulator for red channel
        
        for i in range(3):
            for j in range(3):
                # Convolution with Sharpenning Kernel
                accumulator_b += sharpen_kernel[i, j] * mat[i, j, 0] 
                accumulator_g += sharpen_kernel[i, j] * mat[i, j, 1] 
                accumulator_r += sharpen_kernel[i, j] * mat[i, j, 2] 

        return np.clip([accumulator_b, accumulator_g, accumulator_r], 0, 255) # limiting the output
        

    @staticmethod
    def edge_detect_img(img: np.ndarray): 
       h, w, channels = img.shape # shape of the image
       new_img = np.zeros((h - 2, w - 2, channels), dtype = np.uint8) # creating output img (h - 2, w - 2 due to convolution)
            
       for i in range(1, h - 1):
           for j in range(1, w - 1): 
               mat = img[i - 1 : i + 2, j - 1 : j + 2] # part of image to get convolved
               new_img[i - 1, j - 1] = ImageManip.convolve_sobel_kernel(mat) # placing in the output img
       return new_img

    
    @staticmethod
    def convolve_sobel_kernel(mat: np.ndarray): 
        # Specific Kernels for Detecting Edges
        sobel_kernel_vertical = np.array([[1, 0, -1], 
                                          [1, 0, -1], 
                                          [1, 0, -1]])
        
        sobel_kernel_horizontal = np.array([[1, 1, 1], 
                                            [0, 0, 0], 
                                            [-1, -1, -1]])
        
        sobel_kernel_diag_45_pos = np.array([[1, 1, 0], 
                                             [1, 0, -1], 
                                             [0, -1, -1]])
        
        sobel_kernel_diag_45_neg = np.array([[0, 1, 1], 
                                             [-1, 0, 1], 
                                             [-1, -1, 0]])
        
        accumulator_b_v = 0 # acumulator for blue channel of vertical lines
        accumulator_g_v = 0 # acumulator for green channel of vertical lines
        accumulator_r_v = 0 # acumulator for red channel of vertical lines
        
        accumulator_b_h = 0 # acumulator for blue channel of horizontal lines
        accumulator_g_h = 0 # acumulator for green channel of horizontal lines
        accumulator_r_h = 0 # acumulator for red channel of horizontal lines

        accumulator_b_diag_45_pos = 0 # acumulator for blue channel of diagonal lines with slope 1
        accumulator_g_diag_45_pos = 0 # acumulator for green channel of diagonal lines with slope 1
        accumulator_r_diag_45_pos = 0 # acumulator for red channel of diagonal lines with slope 1
        
        accumulator_b_diag_45_neg = 0 # acumulator for blue channel of diagonal lines with slope -1
        accumulator_g_diag_45_neg = 0 # acumulator for green channel of diagonal lines with slope -1
        accumulator_r_diag_45_neg = 0 # acumulator for red channel of diagonal lines with slope -1

        
        for i in range(3):
            for j in range(3):
                # Convolving with Vertical Kernel
                accumulator_b_v += sobel_kernel_vertical[i, j] * mat[i, j, 0] 
                accumulator_g_v += sobel_kernel_vertical[i, j] * mat[i, j, 1] 
                accumulator_r_v += sobel_kernel_vertical[i, j] * mat[i, j, 2] 
                # Convolving with Horizontal Kernel
                accumulator_b_h += sobel_kernel_horizontal[i, j] * mat[i, j, 0] 
                accumulator_g_h += sobel_kernel_horizontal[i, j] * mat[i, j, 1] 
                accumulator_r_h += sobel_kernel_horizontal[i, j] * mat[i, j, 2] 
                # Convolving with diagonal positive 45 degree Kernel
                accumulator_b_diag_45_pos += sobel_kernel_diag_45_pos[i, j] * mat[i, j, 0] 
                accumulator_g_diag_45_pos += sobel_kernel_diag_45_pos[i, j] * mat[i, j, 1]
                accumulator_r_diag_45_pos += sobel_kernel_diag_45_pos[i, j] * mat[i, j, 2] 
                # Convolving with diagonal negative 45 degree Kernel
                accumulator_b_diag_45_neg += sobel_kernel_diag_45_neg[i, j] * mat[i, j, 0] 
                accumulator_g_diag_45_neg += sobel_kernel_diag_45_neg[i, j] * mat[i, j, 1]
                accumulator_r_diag_45_neg += sobel_kernel_diag_45_neg[i, j] * mat[i, j, 2] 
                

        accumulator_b_final = int((accumulator_b_v ** 2 + accumulator_b_h ** 2 + accumulator_b_diag_45_pos ** 2 + accumulator_b_diag_45_neg ** 2) ** 0.5) # taking sqrt of vertical and horizonal squares
        accumulator_g_final = int((accumulator_g_v ** 2 + accumulator_g_h ** 2 + accumulator_g_diag_45_pos ** 2 + accumulator_g_diag_45_neg ** 2) ** 0.5) # taking sqrt of vertical and horizonal squares
        accumulator_r_final = int((accumulator_r_v ** 2 + accumulator_r_h ** 2 + accumulator_r_diag_45_pos ** 2 + accumulator_r_diag_45_neg ** 2) ** 0.5) # taking sqrt of vertical and horizonal squares

        return np.clip([accumulator_b_final, accumulator_g_final, accumulator_r_final], 0, 255) # limiting the output
        

    @staticmethod
    def gray_scale_convertion(img: np.ndarray):
        h, w, channels = img.shape # shape of the image
        new_img = np.zeros((h, w), dtype = np.single) # creating a new matrix corresponding to scales
        for i in range(h):
            for j in range(w):
                # Calcutating C_srgb
                B_srgb = img[i, j, 0] / 255
                G_srgb = img[i, j, 1] / 255
                R_srgb = img[i, j, 2] / 255

                # Calculating C_linear
                B_linear = None
                G_linear = None
                R_linear = None
                
                if B_srgb <= 0.04045: B_linear = B_srgb / 12.92
                else: B_linear = ((B_srgb + 0.055) / 1.055) ** 2.4

                if G_srgb <= 0.04045: G_linear = G_srgb / 12.92
                else: G_linear = ((G_srgb + 0.055) / 1.055) ** 2.4

                if R_srgb <= 0.04045: R_linear = R_srgb / 12.92
                else: R_linear = ((R_srgb + 0.055) / 1.055) ** 2.4

                # Calculating Y_linear by specific coeficients
                Y_linear = 0.2126 * R_linear + 0.7152 * G_linear + 0.0722 * B_linear
                new_img[i, j] = Y_linear
        return new_img

    @staticmethod
    def color_inversion(img: np.ndarray):
        h, w, channels = img.shape # shape of the image
        new_img = np.zeros((h, w, channels), dtype = np.uint8) # creating a new matrix corresponding to scales
        for i in range(h):
            for j in range(w):
                inverted_b = 255 - img[i, j, 0]
                inverted_g = 255 - img[i, j, 1]
                inverted_r = 255 - img[i, j, 2]
                new_img[i, j] = np.array([inverted_b, inverted_g, inverted_r])
        return new_img

    @staticmethod
    def color_balance(img: np.ndarray): 
        h, w, channels = img.shape # shape of the image
        
        blue = img[:, :, 0]
        green = img[:, :, 1]
        red = img[:, :, 2]

        # Normalize the RGB channels
        red = (red - red.min()) / (red.max() - red.min())
        green = (green - green.min()) / (green.max() - green.min())
        blue = (blue - blue.min()) / (blue.max() - blue.min())

        # Combine the normalized RGB channels
        new_img = np.dstack((red, green, blue))

        # Scale the pixel values back to the range [0, 255]
        new_img = (new_img * 255).astype(np.uint8)
        return new_img
        

# Image Resizing

In [12]:
img = cv2.imread("kuroky.jpg")
cv2.imshow("Original Image", img)
new_img = ImageManip.resize_img(img, 1.5, 1.5)
cv2.imshow("Resized Image", new_img)
cv2.imwrite("resized_kuroky_by_1.5.png", new_img)
cv2.waitKey(0) 
cv2.destroyAllWindows()

![resized Image](./resized_kuroky_by_1.5.png) ![original image](./kuroky.jpg)

# Image Rotation

In [13]:
new_img = ImageManip.rotate_img(img, 45)
cv2.imshow("Rotated Image", new_img)
cv2.imwrite("rotated_kuroky_by_45_deg.png", new_img)
cv2.waitKey(0) 
cv2.destroyAllWindows()

![rotated image by 45 degres](rotated_kuroky_by_45_deg.png) ![original image](kuroky.jpg)

# Image Filtering

## Image Blurring

In [14]:
new_img_1 = ImageManip.blur_img(img, 1)
cv2.imshow("Blurred Image by method 1", new_img_1)
new_img_2 = ImageManip.blur_img(img, 2)
cv2.imshow("Blurred Image by method 2", new_img_2)
cv2.imwrite("blurred_kuroky_1.png", new_img_1)
cv2.imwrite("blurred_kuroky_2.png", new_img_2)
cv2.waitKey(0) 
cv2.destroyAllWindows()

![blurred image](blurred_kuroky_1.png) ![blurred image](blurred_kuroky_2.png) ![original image](kuroky.jpg)

## Sharpening an Image

In [15]:
new_img = ImageManip.sharpen_img(img)
cv2.imshow("Sharpened Image", new_img)
cv2.imwrite("sharpened_kuroky.png", new_img)
cv2.waitKey(0) 
cv2.destroyAllWindows()

![sharpened image](sharpened_kuroky.png) ![original image](kuroky.jpg)

## Edge Detection on an Image

In [ ]:
img = cv2.imread("machine.png")
cv2.imshow("Original Image", img)
new_img = ImageManip.edge_detect_img(img)
cv2.imshow("Edges of the Image", new_img)
cv2.imwrite("edges_machine.png", new_img)
cv2.waitKey(0) 
cv2.destroyAllWindows()

![image after edge detection](edges_machine.png) ![original image](machine.png)

# Color Manipulation

## Gray Scale Convertion

In [4]:
img = cv2.imread("machine.png")
cv2.imshow("Original Image", img)
new_img = ImageManip.gray_scale_convertion(img)
gray_image = cv2.cvtColor(new_img, cv2.COLOR_GRAY2BGR)
cv2.imshow("Gray Scale of Image", gray_image)
result = cv2.normalize(gray_image, dst=None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
cv2.imwrite("gray_scale_machine.jpg", result)
cv2.waitKey(0) 
cv2.destroyAllWindows()

![gray scale of image](gray_scale_machine.jpg) ![original image](machine.png)

## Color Inversion 

In [6]:
img = cv2.imread("machine.png")
cv2.imshow("Original Image", img)
new_img = ImageManip.color_inversion(img)
cv2.imshow("Color Inversion", new_img)
cv2.imwrite("color_inversion_machine.png", new_img)
cv2.waitKey(0) 
cv2.destroyAllWindows()

![color inversion](color_inversion_machine.png) ![original image](machine.png)

## Color Balance

In [ ]:
img = cv2.imread("hillside.png")
cv2.imshow("Original Image", img)
new_img = ImageManip.color_balance(img)
cv2.imshow("Color Balanced", new_img)
cv2.imwrite("color_balance_hillside.png", new_img)
cv2.waitKey(0) 
cv2.destroyAllWindows()

![color balanced](color_balance_hillside.png) ![original image](hillside.png)

# Extra:
- 2 non-trivial approaches to blur image (using gaussian kernel)
- considering 4 different cases for a line to get detected
- non-trivial approach for gray-scaling
- non-trivial approach for color balancing
- efficient approaches used for all transformations as you can see in each part